# Exploration of the dataset and testing notebook

### Explore the dataset to check categories, missing values etc (important for preprocessing)

In [5]:
import pandas as pd
df = pd.read_csv("../data/raw/filtered_final_cleaned_data.csv")
df.head()


,property_ID,locality_name,postal_code,type,subtype,price (€),number_of_bedrooms,living_area (m²),"equiped_kitchen (yes:1, no:0)","furnished (yes:1, no:0)","open_fire (yes:1, no:0)","terrace (yes:1, no:0)",terrace_area (m²),"garden (yes:1, no:0)",number_facades,"swimming_pool (yes:1, no:0)",state_of_building,province
0,RBU60880,Maurits Sabbestraat 4 202,2800,Apartment,Apartment,329000,3.0,104,0,0,0,1,NaN,0,2.0,0,Excellent,Antwerp
1,RBU61001,Nieuwstraat 13,2200,House,Residence,425000,3.0,378,0,0,0,1,NaN,1,2.0,0,To be renovated,Antwerp
2,RBU62593,Veerstraat,2840,Apartment,Apartment,264700,1.0,69,0,0,0,1,25.0,0,NaN,0,NaN,Antwerp
3,RBU60705,Winkelomseheide 158,2440,Apartment,Apartment,290000,2.0,95,0,0,0,1,15.0,1,2.0,0,New,Antwerp
4,RBU60944,Generaal van der Meerschstraat 85 2,2300,Apartment,Apartment,180000,2.0,88,0,0,0,1,2.0,0,2.0,0,Normal,Antwerp


In [6]:
#Overview of all the columns and the total missing values per column
df.isna().sum()

property_ID                         0
locality_name                    2328
postal_code                         0
type                                0
subtype                             0
price (€)                           0
number_of_bedrooms                148
living_area (m²)                    0
equiped_kitchen (yes:1, no:0)       0
furnished (yes:1, no:0)             0
open_fire (yes:1, no:0)             0
terrace (yes:1, no:0)               0
terrace_area (m²)                6750
garden (yes:1, no:0)                0
number_facades                   4197
swimming_pool (yes:1, no:0)         0
state_of_building                2913
province                            0
dtype: int64

In [7]:
# see the categories and total count per category for "subtype" and "state_of_building" column
df["subtype"].value_counts()
df["state_of_building"].value_counts()

state_of_building
New                   5748
Normal                2657
Excellent             1312
To be renovated       1215
To renovate            404
Fully renovated        275
Under construction       8
To restore               8
To demolish              5
Name: count, dtype: int64

In [8]:

# get categorical columns:
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
categorical_cols

# get numerical columns:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
numeric_cols

['postal_code',
 'price (€)',
 'number_of_bedrooms',
 'living_area (m²)',
 'equiped_kitchen (yes:1, no:0)',
 'furnished (yes:1, no:0)',
 'open_fire (yes:1, no:0)',
 'terrace (yes:1, no:0)',
 'terrace_area (m²)',
 'garden (yes:1, no:0)',
 'number_facades',
 'swimming_pool (yes:1, no:0)']

### Add the province column back to filtered_final_cleaned_data.csv

In [6]:
# Load dataset
import pandas as pd

"""We need to add a "province" column to our filtered_final_cleaned_data.csv file. 
Since we only have postal codes, we'll first need to map them to their provinces. 
We'll use a dictionary mapping postal codes to provinces."""

# Load data
df = pd.read_csv("../data/raw/filtered_final_cleaned_data.csv")

# Define postal code ranges per province
postal_to_province = {
    "Antwerp": range(2000, 3000),
    "East-Flanders": range(9000, 10000),
    "West-Flanders": range(8000, 9000),
    "Flemish-Brabant": list(range(1500, 2000)) + list(range(3000, 3500)),
    "Brussels": range(1000, 1300),
    "Limburg": range(3500, 4000),
    "Liège": range(4000, 5000),
    "Namur": range(5000, 6000),
    "Hainaut": list(range(6000, 6600)) + list(range(7000, 8000)),
    "Luxembourg": range(6600, 7000),
    "Brabant-Wallon": range(1300, 1500)
}

# Helper function to find province for each postal code
def get_province(postal_code):
    try:
        postal_code = int(postal_code)
        for province, codes in postal_to_province.items():
            if postal_code in codes:
                return province
        return "Unknown"
    except:
        return "Unknown"


# Apply the function to create a new column called "province" with the province names based on the postal codes.
df["province"] = df["postal_code"].apply(get_province)

print(df[["postal_code", "province"]].head())

# display(df.head())

# Save the updated dataset
df.to_csv("../data/raw/filtered_final_cleaned_data.csv", index=False)
print("CSV saved with 'province' column!")

# Load the new dataset with province column
df = pd.read_csv("../data/raw/filtered_final_cleaned_data.csv")






   postal_code province
0         2800  Antwerp
1         2200  Antwerp
2         2840  Antwerp
3         2440  Antwerp
4         2300  Antwerp
CSV saved with 'province' column!


In [ ]:
# One hot encoding function

### Bring together and see your fully preprocessed dataset

All preprocessing steps are applied to X_train and X_test, not the original df.
So the original df will remain:
with NaNs
not encoded
not scaled
This is expected — sklearn never modifies the original DataFrame.

Most sklearn transformers (OneHotEncoder, StandardScaler, etc.) output NumPy arrays, which:
don’t keep column names,
don’t automatically merge back into X_train,
won’t be visible in df.head().
Unless you explicitly rebuild a DataFrame with the transformed results, nothing changes.

How to see your fully preprocessed dataset:   
you must manually combine the transformed arrays back into a DataFrame:


Option A — Using ColumnTransformer + Pipeline

In [ ]:
# Fit on train
X_train_processed = preprocessor.fit_transform(X_train)

# Transform test
X_test_processed = preprocessor.transform(X_test)

# Turn into DataFrame
processed_cols = preprocessor.get_feature_names_out()

X_train_final = pd.DataFrame(X_train_processed, columns=processed_cols, index=X_train.index)
X_test_final = pd.DataFrame(X_test_processed, columns=processed_cols, index=X_test.index)

X_train_final.head()


Option B — doing everything manually:

In [ ]:
# 1. Impute
X_train["living_area"] = imputer.fit_transform(X_train[["living_area"]])
X_test["living_area"] = imputer.transform(X_test[["living_area"]])

# 2. Encode
ohe = OneHotEncoder(sparse_output=False)
encoded = ohe.fit_transform(X_train[["province"]])
cols = ohe.get_feature_names_out(["province"])
X_train_encoded = pd.DataFrame(encoded, columns=cols, index=X_train.index)

# 3. Scale
scaler = StandardScaler()
scaled = scaler.fit_transform(X_train[["living_area"]])
X_train_scaled = pd.DataFrame(scaled, columns=["living_area_scaled"], index=X_train.index)

# 4. Combine everything
X_train_final = pd.concat([X_train, X_train_encoded, X_train_scaled], axis=1)
